# From Docplex to QUBO representation 

The Decision Optimization CPLEX modeling for python ([DOCPLEX](https://pypi.org/project/docplex/)) is a library that allows smooth prototyping for optimization problems. Here, we present the converter ```FromDocplex2IsingModel``` that translates the DOCPLEX optimization models into its OpenQAOA [QUBO](https://en.wikipedia.org/wiki/Quadratic_unconstrained_binary_optimization#Connection_to_Ising_models) representation. In general, problems in the form of [quadratic programming](https://en.wikipedia.org/wiki/Quadratic_programming) can be encoded using QUBOs. The three ingredints of an optimization model are the objective function, the equality constraints, and the inequality constraints. 

In this example, we show the solution of:

    1. A basic optimization model using minimize.

    2. A basic optimization model using maximize.


In [1]:
%matplotlib notebook

# Import external libraries to present an manipulate the data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import docplex model to generate the problem to optimize
from docplex.mp.model import Model

# Import the libraries needed to employ the QAOA quantum algorithm using OpenQAOA
from openqaoa.workflows.optimizer import QAOA

# method to covnert a docplex model to a qubo problem
from openqaoa.problems.converters import FromDocplex2IsingModel
from openqaoa.devices import create_device

# method to find the corrects states for the QAOA boject 
from openqaoa.utilities import ground_state_hamiltonian

## 1) A basic optimization problem using minimize

Representation of a quadratic programming problem includes the cost function, the equality constraints, and the inequality constraints. Imagine that you want to minimize the follwing objective function $f_1(x)$

$$f_1(\textbf{x}) = x_0 + 3 x_0 x_1 + x_0 x2 - 5 x_4 x_2 + 3 x_4 x_3 + 3 x_4 $$

subject to the following equality constraint

$$x_0 + 3 x_1 + 2 x_4 = 3,$$

and the inequality constraint:

$$x_2 + 3 x_3 + 2 x_4 \ge 3.$$

First, this problem is created from a model in `DOCPLEX`.  Using OpenQAOA such a DOCPLEX model can be converted into a QUBO and the QUBO can be solved using QAOA.

In [2]:
#Specific the Model and put a name
mdl = Model("Basic Problem using minimize")

Adding the binary variables of the problem

In [3]:
# Number of variables
n_vars = 5 

# Using binary variables for this model
x = mdl.binary_var_list(n_vars, name="x") 

- 1 Adding the objective function

In [4]:
# Design the objective function using the binary variables
obj_func = x[0] + 3 * x[0] * x[1] + x[0] * x[2] - 5 * x[4] * x[2] + 3 * x[4] * x[3] + 3 * x[4]

# The problem could be minimize or maximize the objective function
mdl.minimize(obj_func)

- 2. Adding equality constraints 

In [5]:
# In Docplex is possible design equality constraints
mdl.add_constraint(x[0] + 3 * x[1] + 2 * x[4] == 3)

docplex.mp.LinearConstraint[](x_0+3x_1+2x_4,EQ,3)

- 3. Adding inequality constraints

In [6]:
# In Docplex is possible design inequality constraints
mdl.add_constraint(x[2] + 3 * x[3] + 2 * x[4] >= 3)

# print a summary of the docplex model 
print(mdl.prettyprint())

// This file has been generated by DOcplex
// model name is: Basic Problem using minimize
// var contrainer section
dvar bool x[5];

minimize
 x_0 + 3 x_4 [ 3 x_0*x_1 + x_0*x_2 - 5 x_2*x_4 + 3 x_3*x_4 ];
 
subject to {
 x_0 + 3 x_1 + 2 x_4 == 3;
 x_2 + 3 x_3 + 2 x_4 >= 3;

}
None


### 1.1 Solving the problem using OpenQAOA

OpenQAOA has the class `FromDocplex2IsingModel`, to convert the DOCPLEX model into a QUBO representation that is the input from the QAOA object.

In [7]:
# Converting the Docplex model into its qubo representation
qubo = FromDocplex2IsingModel(mdl)

# Ising encoding of the QUBO problem
ising_encoding = qubo.ising_model  

# Print in a df the ising encoding
pd.DataFrame(ising_encoding.asdict())

,terms,weights,constant,_n
0,"[0, 1]",26.25,181.0,7
1,"[0, 2]",0.25,181.0,7
2,"[2, 4]",15.75,181.0,7
3,"[3, 4]",51.75,181.0,7
4,"[0, 4]",17.00,181.0,7
5,"[1, 4]",51.00,181.0,7
6,"[2, 3]",25.50,181.0,7
7,"[2, 5]",-8.50,181.0,7
8,"[2, 6]",-17.00,181.0,7
9,"[3, 5]",-25.50,181.0,7


For this problem you can generate a device that uses a specific backend, such as the qiskit's qasm_simulator.

In [8]:
#Specific local device usign qiskit backend
device = create_device("local", 'qiskit.qasm_simulator')

A QAOA object is initialized to use the quantum algorithm with the device and is possible change some properties: number of shots, seed, p-value, init type, and variational params.

In [9]:
#Is possible check the devices using qaoa.local_simulators, qaoa.cloud_provider
qaoa = QAOA(device)

#Indicate the properties to the QAOA quantum algorithm,shots,seed
qaoa.set_backend_properties(n_shots=20, seed_simulator=1)
#check the p value and the variational init params
qaoa.set_circuit_properties(p=2, init_type="custom", variational_params_dict={"betas":2*[0.01*np.pi],"gammas":2*[0.01*np.pi]})

#Indicate the ising e ncoding model from docplex 
qaoa.compile(ising_encoding)

#Optimize the quantum algorithm
qaoa.optimize()

We can limit the number of states from the `lowest_cost_bitstrings` method and indicate the 5 best solutions of the problem with their respective energies and probabilities by using QAOA

In [10]:
#Print in a df the best 5 solutions
pd.DataFrame(qaoa.results.lowest_cost_bitstrings(5))

,solutions_bitstrings,bitstrings_energies,probabilities
0,0111010,0.0,0.05
1,0101000,0.0,0.05
2,1010100,0.0,0.05
3,0010100,15.0,0.05
4,0111000,17.0,0.10


From the solution we can see that we obtain three states `0111010`, `1010100`, and `0101000` with the minimum energy `0`. this can be confirmed  using the `ground_state_hamiltonian` method.

In [11]:
# To find the correct answer using ground_state_hamiltonian
# and  the paremeter is a cost_hamiltonian
correct_solution = ground_state_hamiltonian(qaoa.cost_hamil)
correct_solution

(0.0, ['0101000', '1010100', '0111010'])

### 1.2 Solution using DOCPLEX

The problem can be solved classically using DOCPLEX, this is an excellent option to check the solution of our quantum solver.

**Note: For the next cell you  will need to install cplex with the command `pip install cplex>=22.1.0.0` **

In [12]:
# Docplex QUBO model
mdl_qubo = qubo.qubo_docplex 

# Obtain the docplex solution
sol = mdl_qubo.solve()
mdl_qubo.print_solution(print_zeros=True)

objective: 0.000
  x_0=0
  x_1=1
  x_2=0
  x_3=1
  x_4=0
  slack_C1_0=0
  slack_C1_1=0


The solution is `0101000`, being one of the quantum solution states.

## 2) A basic optimization problem using maximize


Considering the previous section, you want to maximize the follwing objective function $f_2(x)$

$$f_2(\textbf{x}) = x_0 - 4 x_0 x_1 - 2 x_0 x2 +  x_1 x_2 - x_3 $$

subject to the following equality constraint

$$ x_2 - 3 x_3 = 1,$$

and the inequality constraint:

$$x_0 + 2 x_2 - 2 x_3 \leq 4.$$



In [13]:
#Specific the Model and put a name
mdl = Model("Basic Problem using maximize")

# Number of variables
n_vars = 4 

# Using binary variables for this model
x = mdl.binary_var_list(n_vars, name="x") 

# Design the objective function using the binary variables
obj_func = x[0] - 4 * x[0] * x[1] - 2* x[0] * x[2] + x[1] * x[2]- x[3]

# The problem could be minimize or maximize the objective function
mdl.maximize(obj_func)

# In Docplex is possible design equality constraints
mdl.add_constraint(x[2] - 3 * x[3]  == 1)

# In Docplex is possible design inequality constraints
mdl.add_constraint(x[0] + 2 * x[2] - 2 * x[3] <= 4)

# print a summary of the docplex model 
print(mdl.prettyprint())

// This file has been generated by DOcplex
// model name is: Basic Problem using maximize
// var contrainer section
dvar bool x[4];

maximize
 x_0 - x_3 [ - 4 x_0*x_1 - 2 x_0*x_2 + x_1*x_2 ];
 
subject to {
 x_2 - 3 x_3 == 1;
 x_0 + 2 x_2 - 2 x_3 <= 4;

}
None


### 2.1 Solving the problem using OpenQAOA

Using the  class `FromDocplex2IsingModel`, to convert the DOCPLEX model into a QUBO representation that is the input from the QAOA object.

In [14]:
# Converting the Docplex model into its qubo representation
qubo = FromDocplex2IsingModel(mdl)

# Ising encoding of the QUBO problem
ising_encoding = qubo.ising_model  

# Print in a df the ising encoding
pd.DataFrame(ising_encoding.asdict())

,terms,weights,constant,_n
0,"[0, 1]",1.00,126.25,7
1,"[0, 2]",10.50,126.25,7
2,"[1, 2]",-0.25,126.25,7
3,"[2, 3]",-35.00,126.25,7
4,"[0, 3]",-10.00,126.25,7
5,"[0, 4]",5.00,126.25,7
6,"[0, 5]",10.00,126.25,7
7,"[0, 6]",15.00,126.25,7
8,"[2, 4]",10.00,126.25,7
9,"[2, 5]",20.00,126.25,7


The device for this problem uses the pyquil's statevector_simulator backend. And use an QAOA object with their respective parameters indicated in the same way as in the previous section. And check only the 5 best states.

In [15]:
#Specific local device usign qiskit backend
device = create_device("local", 'pyquil.statevector_simulator')

#Is possible check the devices using qaoa.local_simulators, qaoa.cloud_provider
qaoa = QAOA(device)

#Indicate the properties to the QAOA quantum algorithm,shots,seed
qaoa.set_backend_properties(n_shots=20, seed_simulator=1)
#check the p value and the variational init params
qaoa.set_circuit_properties(p=1, init_type="custom", variational_params_dict={"betas":[0.01*np.pi],"gammas":[0.01*np.pi]})

#Indicate the ising e ncoding model from docplex 
qaoa.compile(ising_encoding)

#Optimize the quantum algorithm
qaoa.optimize()

#Print in a df the best 5 solutions
pd.DataFrame(qaoa.results.lowest_cost_bitstrings(5))

,solutions_bitstrings,bitstrings_energies,probabilities
0,0110010,-1.0,0.025332
1,0010010,0.0,0.022416
2,1010100,1.0,0.007213
3,1110100,4.0,0.009391
4,0110110,9.0,0.027665


The solution is the  state `0110010`  with the minimum energy -1. this can be confirmed using the ground_state_hamiltonian method.

In [16]:
# To find the correct answer using ground_state_hamiltonian
# and  the paremeter is a cost_hamiltonian
correct_solution = ground_state_hamiltonian(qaoa.cost_hamil)
correct_solution

(-1.0, ['0110010'])

### 2.2 Solution using DOCPLEX

Using DOCPLEX to check the classical solution

**Note: For the next cell you  will need to install cplex with the command** `pip install cplex>=22.1.0.0` 

In [17]:
# Docplex QUBO model
mdl_qubo = qubo.qubo_docplex 

# Obtain the docplex solution
sol = mdl_qubo.solve()
mdl_qubo.print_solution(print_zeros=True)

objective: -1.000
  x_0=0
  x_1=1
  x_2=1
  x_3=0
  slack_C1_0=0
  slack_C1_1=1
  slack_C1_2=0


The solution is `0110010`, being the same result as for the quantum algorithm